In [71]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [72]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'a') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [73]:
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

In [74]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [75]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [76]:
BASELINE_GENERATE_TC = """
read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":{
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
  },
}
If there are more than one test case for this scenario, continue writing other test case in this form."""

In [77]:
use_case_directory_path = r"D:\Dissertation-GPT\dataset\SpecificationData\Shopee(new)"
save_path = r"D:\Dissertation-GPT\evaluate\ResultSet\6.24\shopee"

In [78]:
usecase_name_list = []
project_name = "shopee"

In [79]:
for filename in os.listdir(use_case_directory_path):
    usecase_name_list.append (filename.split(".txt")[0])
print(usecase_name_list)

['add to cart', 'checkout', 'product information display', 'purchase', 'search']


In [80]:
test_scenario_directory_path = r"D:\Dissertation-GPT\evaluate\ResultSet\6.24\shopee (baseline)"
save_testcase_path = r"D:\Dissertation-GPT\evaluate\baseline 2\shopee\tc"

In [81]:

for usecase in usecase_name_list:
  base_name = os.path.basename(usecase)
  usecase_name =usecase
  usecase_path = os.path.join (use_case_directory_path,f"{usecase_name}.txt" )
  testscenario_path = os.path.join (test_scenario_directory_path,f"{usecase_name}.txt" )
  testscenario_list = read_file_to_list(testscenario_path)
  print(testscenario_path)
  print(testscenario_list)
  for testscenario in testscenario_list:
    content = "Test scenarios:"+ testscenario+  "\nUse case:" + read_file_content(usecase_path)
    print(content)
    promptExtractCondition = [
      { "role": "system", "content": BASELINE_GENERATE_TC},
      { "role": "user", "content": content}
    ]
    gpt_response = askJSON(promptExtractCondition, client, model)
    print(gpt_response)
    print()
    json_data = json.loads(gpt_response)
    testscenario = testscenario.replace('"','').replace(':','').replace('?','').replace('*','').replace('/','')
    testcase_path = os.path.join (save_testcase_path,f"{usecase_name}-{testscenario}.txt" )
    write_json_to_txt_raw(json_data,testcase_path )
  print("f{usecase_name} gen test case complete")

D:\Dissertation-GPT\evaluate\ResultSet\6.24\shopee (baseline)\add to cart.txt
['1. AddSingleItemToCart', '2. AddItemWithMultipleOptionsToCart', '3. AdjustQuantityAndAddToCart', '4. AddItemWithoutSelectingOption', '5. AddItemWithQuantityMoreThanStock', '6. DecreaseQuantityBelowOne', '7. AddOutOfStockItemToCart', '8. AddItemWithOutOfStockOptionToCart', '9. ViewCartAfterAddingItem', '10. UpdateCartIconAfterAddingItem', '11. AddMultipleDifferentItemsToCart', '12. AddSameItemMultipleTimesToCart', '13. AddItemToCartWithoutLogin', '14. AddItemToCartAndContinueShopping', '15. AddItemToCartAndViewCartDetails']
Test scenarios:1. AddSingleItemToCart
Use case:Use case name: Add item to cart
Description: This feature allows users to add items to their shopping cart for future purchase.
Actor: User
Preconditions: User is logged into their Shopee account.
Postconditions: 
The item is successfully added to the user's shopping cart.
The user can view and manage items in their shopping cart.
Basic Flow: